In [ ]:
# Network interfaces
    
class CostFunction:
    """Define the cost function interface of the network
    """
    def calculate(self, value : list[float], predicted_value : list[float]) -> float:
        pass

    def calculate_d(self, value : list[float], predicted_value : list[float]) -> float:
        pass
    
class ActivationFunction:
    """Define the activation function interface of the network
    """
    def activation(self, x : list[float]) -> list[float]:
        """Define the activation function

        Args:
            x (list[float]): input values

        Returns:
            list[float]: output values
        """
        pass
    
    def activation_d(self, x : list[float]) -> list[float]:
        """Define the derivative of the activation function

        Args:
            x (list[float]): input values

        Returns:
            list[float]: output values
        """
        pass
    
class Layer:
    """Define the layer interface of the network
    """
    def __init__(self, cost_function : CostFunction):
        self.input , self.output = None, None
        self.cost_function = cost_function

    def forward(self, input_value : list[float]) -> list[float]:
        """Define the forward propagation of the layer

        Args:
            input_value (float): input values

        Returns:
            float: _description_
        """
        pass

    def backward(self, error : list[float], learning_rate : float) -> list[float]:
        """Define the backward propagation of the layer

        Args:
            error (list[float]): error of the previous layer
            learning_rate (float): learning rate

        Returns:
            list[float]: the errors of the next layer
        """
        pass

In [ ]:
class Neuron:
    def __init__(self, input_indices : list[int], activation_function : ActivationFunction):
        self.weights = np.random.rand(len(input_indices)) - 0.5
        self.bias = np.random.random() - 0.5
        self.input_indices = input_indices
        
        self.last_output = None
        self.last_input = None
        self.delta = None
        
        self.activation_function = activation_function

    def forward(self, inputs):
        self.last_input = inputs
        
        weighted_sum = np.dot(inputs[self.input_indices], self.weights) + self.bias
        output = self.activation_function(weighted_sum)
        
        self.last_output = output
        return output

    def backward(self, output_error, learning_rate):   
        self.delta = output_error * self.activation_function.activation_d(self.last_output)
        
        # Update the weights and bias
        self.weights += learning_rate * np.dot(self.delta, self.last_input[self.input_indices])
        self.bias += learning_rate * self.delta

    def get_input_indices(self):
        return self.input_indices

In [ ]:
# Connection layers

class FullyConnectedLayer(Layer):
    
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    def forward(self, input_data):
        self.input = input_data
        return np.dot(self.input, self.weights) + self.bias

    def backward(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

class NotFullyConnectedLayer(Layer):
    def __init__(self, cost_function: CostFunction, input_indices: list[list[int]]):
        super().__init__(cost_function)
        self.input_indices = input_indices
        self.neurons = [Neuron(indices) for indices in input_indices]

    def forward(self, input_value: list[float]) -> list[float]:
        self.input = input_value
        outputs = [neuron.forward(input_value) for neuron in self.neurons]
        self.output = outputs
        return outputs

    def backward(self, error: list[float], learning_rate: float) -> list[float]:
        prev_layer_error = np.zeros(len(self.input))

        for i, neuron in enumerate(self.neurons):
            neuron_error = error[i]
            prev_layer_error[neuron.input_indices] += neuron.backward(neuron_error, learning_rate, self.input)

        return prev_layer_error

class ActivationLayer(Layer):
    def __init__(self, activation_function : ActivationFunction):
        self.activation_function = activation_function

    def forward(self, input_data):
        self.input = input_data
        return self.activation_function.activation(input_data)

    def backward(self, output_error, learning_rate):
        return self.activation_function.activation_d(self.input) * output_error